In [65]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

data = fetch_movielens(min_rating = 4.0) # 

#print(repr(data['train']))
#print(repr(data['test']))

model = LightFM(loss = 'warp')
model.fit(data['test'], epochs=35, num_threads=2)

def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()                                    
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels']
        top_items_original = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:5]:
            print("        %s" % x)
        
        print("     Recommended:")
        
        for x in top_items_original[:3]:
           print("        %s" % x)
        print()
        #print(scores.shape)
        #print(top_items.shape)
        mat = np.column_stack((top_items,scores))
        mat_sort = mat[np.argsort(-scores)]
        for x in mat_sort[:5]:
            print("        %s" % x)
        print()
        #for x in mat[:3]:
        #    print("        %s" % x)


In [1]:
sample_recommendation(model, data, [1,2,3])

NameError: name 'sample_recommendation' is not defined